In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import yaml

import ixmp
import nomenclature
import pyam

In [ ]:
eu_climate = ixmp.Platform("eu-climate-submission")

In [ ]:
eu_dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
countries = []

for country in eu_dsd.region:
    if eu_dsd.region[country].hierarchy == "country":
        if eu_dsd.region[country].eu_member:
            countries.append(country)

In [ ]:
scenarios = eu_climate.scenario_list()

In [ ]:
list()

In [ ]:
meta = (
    pd.DataFrame(scenarios[["model", "scenario", "annotation"]])
    .set_index(["model", "scenario"])
)

In [ ]:
attributes = ["source", "reference", "doi",
              "coverage: global", "coverage: R5", "coverage: R10",
              "coverage: EU27 & UK", "coverage: EU27", "coverage: country"]

In [ ]:
for a in attributes:
    meta[a] = None

In [ ]:
# translate to CodeList-like dictionary 

with open("scenario_metadata.yaml") as f:
    _scenario_metadata = yaml.safe_load(f)

scenario_metadata = dict()
for model_element in _scenario_metadata:
    for model, scenario_list in model_element.items():
        _scenario_dict = dict()
        for scenario in scenario_list:
            for name, attrs in scenario.items():
                _scenario_dict[name] = attrs
        scenario_metadata[model] = _scenario_dict

In [ ]:
# assign metadata-attributes to meta table
for model, _df in scenarios.groupby("model"):
    for scenario_group in scenario_metadata[model]:

        model_scenarios = _df.scenario[pyam.pattern_match(_df.scenario, scenario_group)]
        if model_scenarios.empty:
            raise ValueError(model + "|" + scenario_group)
        for s in _df.scenario[pyam.pattern_match(_df.scenario, scenario_group)]:
            run = ixmp.Scenario(eu_climate, model=model, scenario=s)

            for a in attributes:
                if a in scenario_metadata[model][scenario_group]:
                    value = scenario_metadata[model][scenario_group][a]
                    if value is not None:
                        meta.loc[(model, s), a] = value
                        run.set_meta(a, value)

In [ ]:
for model, _df in scenarios.groupby("model"):
    for scenario in list(_df.scenario):
        run = ixmp.Scenario(eu_climate, model=model, scenario=scenario)
        df = run.timeseries()
        regions = list(df.region.unique())
        if "World" in regions:
            meta.loc[(model, scenario), "coverage: global"] = True
            run.set_meta("coverage: global", True)
        if "OECD & EU (R5)" in regions:
            meta.loc[(model, scenario), "coverage: R5"] = True
            run.set_meta("coverage: R5", True)
        if "Europe (R10)" in regions:
            meta.loc[(model, scenario), "coverage: R10"] = True
            run.set_meta("coverage: R10", True)
        if "EU27 & UK" in regions:
            meta.loc[(model, scenario), "coverage: EU27 & UK"] = True
            run.set_meta("coverage: EU27 & UK", True)
        if "EU27" in regions:
            meta.loc[(model, scenario), "coverage: EU27"] = True
            run.set_meta("coverage: EU27", True)
        if set(regions).issubset(countries):
            meta.loc[(model, scenario), "coverage: country"] = True
            run.set_meta("coverage: country", True)

        print(f"{model} - {scenario}")

In [ ]:
meta.to_excel("eu_cab_meta_refs.xlsx")

In [ ]:
meta.loc["REMIND-MAgPIE 2.1-4.2"]

In [ ]:
meta = pd.read_excel("eu_cab_meta_refs.xlsx").set_index(["model", "scenario"])

In [ ]:
meta.columns

In [ ]:
columns = ['source', 'reference', 'doi']
coverage_columns = ['coverage: global', 'coverage: R5', 'coverage: R10', 'coverage: EU27 & UK', 'coverage: EU27', 'coverage: country']

In [ ]:
list()

In [ ]:
for (_model, scenario), df in meta.iterrows():
    print(model, scenario, df["doi"])

In [ ]:
model = None

for (_model, scenario), df in meta.iterrows():
    if not pd.isna(_model):
        model = _model

    run = ixmp.Scenario(eu_climate, model=model, scenario=scenario)
    for c in columns:
        if not pd.isna(df[c]):
            run.set_meta(c, df[c])
    for c in coverage_columns:
        if df[c]:
            run.set_meta(c, True)
    print(model, scenario)

In [ ]:
pd.isna(df["doi"])

In [ ]:
meta[pd.isna(meta.source)]

In [ ]:
scenario_metadata[model].keys()

In [ ]:
scenario_metadata[model]

In [ ]:
list()

In [ ]:
scenario_metadata